# Random Forest

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("dataset/CL_over-dataset.csv",sep='\t',decimal=",",index_col=0)

In [18]:
df.head()

,Imax,E,PIL,Mt,TsW,NbxW,MeanD,Eday_of_week,class
0,12,4.335643,2.567,18.202218,100.627358,0.641509,0.000000,0.845790,0
1,32,6.503112,2.567,18.970947,60.492264,0.169811,35.000000,1.942764,1
2,360,6.504979,2.752,28.764242,125.368679,0.283019,24.266667,2.396410,1
3,80,4.494680,2.567,33.866071,17.891509,0.094340,55.400000,1.177965,0
4,32,0.000000,2.567,292.000000,16.528302,0.056604,13.000000,0.918296,1


In [19]:
from sklearn.model_selection import train_test_split
label = df.pop('class')
train_set, test_set, train_label, test_label = train_test_split(df, label, stratify =label, test_size=0.20)

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


## Grid search

In [21]:


param_dist = [{"max_depth": [2,3,12,None],
              "max_features": [5],# Al massimo 6 non di più
              "min_samples_split": [10,50],
              "min_samples_leaf": [10,50],
              "bootstrap": [True, False],
              "criterion": ["entropy", "gini"],
              "class_weight":['balanced', None, {0: 0.3, 1: 0.7}]}]

clf = RandomForestClassifier(n_estimators=30)
grid_search = GridSearchCV(clf, param_dist,  
                            n_jobs=3, 
                            scoring='accuracy')

grid_search.fit(train_set, train_label)

print("Complete Grid search:")
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

print("Best parameters:",grid_search.best_params_)
print("Best Score: ",grid_search.best_score_)


leaf': 10, 'min_samples_split': 10}
0.613 (+/-0.049) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.618 (+/-0.072) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 10}
0.608 (+/-0.060) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 2, 'max_features': 5, 'min_samples_leaf': 50, 'min_samples_split': 50}
0.729 (+/-0.023) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 10}
0.724 (+/-0.008) for {'bootstrap': True, 'class_weight': {0: 0.3, 1: 0.7}, 'criterion': 'gini', 'max_depth': 3, 'max_features': 5, 'min_samples_leaf': 10, 'min_samples_split': 50}
0.731 (+/-0.023) for {'bootstrap': True, 'class_weight': {0: 0.3

## Modello Finale

In [22]:
rf = RandomForestClassifier(n_estimators=30, 
                             criterion='entropy',
                             class_weight=None,
                             max_features=5,
                             max_depth=12, 
                             min_samples_split=50,
                             min_samples_leaf=10,
                             bootstrap=True) 
rf = rf.fit(train_set, train_label)

In [23]:
#prediction on the test test
test_pred_rf = rf.predict(test_set)

In [24]:
#compute the performance of the model
print(classification_report(test_label, 
                            test_pred_rf))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       564
           1       0.96      0.95      0.95       564
           2       0.97      0.99      0.98       564

    accuracy                           0.97      1692
   macro avg       0.97      0.97      0.97      1692
weighted avg       0.97      0.97      0.97      1692

